# Modeling Part 1 -- Linear Regression
---

With the data now cleaned, we are going to start simple with a linear regression model to build a performance baseline. Due to the nature of our data being almost entirely categorical/ordinal, we do not expect this to perform well, even with Lasso (L1) regularization.

Our baseline will be calculated using `api_delta` only (calculated difference of API score between post and pre treatment).

To make life easier, we will use the data that does not included missing values.


## Import Packages + Setup

In [130]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools

# SKLEARN
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, mutual_info_regression 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# stats models
import statsmodels.api as sm
from scipy import stats

In [131]:
df = pd.read_csv("../data/cleaned/not_scaled_no_missing.csv")

In [132]:
# Making all columns numeric again 
# Writing to CSV sometimes mucks with data types :(
cols = df.columns

df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

In [133]:
# expect 184 records if dropping all NAs
df.shape

(184, 129)

In [134]:
# Create target dataframe (difference between scores)
df_target = df[['study_group', 'cluster2019_base']]
df_target['api_delta'] = df['api_tot_recode0_4_post'] - df['api_tot_recode0_4_base']
df_target['csigi_delta'] = df['csigimn2019_post'] - df['csigimn2019_base']
df_target['cprinter_delta'] = df['cprinterf8_post'] - df['cprinterf8_base']

# Drop 
df = df.drop(
    ['api_tot_recode0_4_post', 'csigimn2019_post', 'cprinterf8_post', 'cprint8_tscore_post',
     'cluster2019_base', 'api_tot_recode0_4_base', 'csigimn2019_base',
    'cprinterf8_base', 'cprint8_tscore_base'
    ],

    axis = 1
)

ipykernel_launcher:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Linear Regression (statsmodels)
---

Using mean absolute error (MAE) becuase the difference from pre-to-post score can be a decimal

In [135]:
X_train, X_test, y_train, y_test = train_test_split(
    df, 
    df_target['api_delta'],
    test_size=0.25
)

In [136]:
# Train Linear Regression Model

# Add intercept + rename
# ref: https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS
sm_reg = sm.add_constant(X_train)
sm_reg = sm_reg.rename(columns={'const': 'intercept'})

est = sm.OLS(y_train, sm_reg)
model_linReg = est.fit()

In [137]:
## Make Predictions
X_test = sm.add_constant(X_test)
predictions = model_linReg.predict(X_test)

In [138]:
# MSE 
sm_mse = ((y_test - predictions)**2).sum() / len(predictions)
sm_mae = abs(y_test - predictions).sum() / len(predictions)

print(f'MSE (statsmodels) -- {round(sm_mse, 2)}')
print(f'MAE (statsmodels) -- {round(sm_mae, 2)}')

MSE (statsmodels) -- 3.12
MAE (statsmodels) -- 1.32


In [139]:
# Model Summary
print(model_linReg.summary())

                            OLS Regression Results                            
Dep. Variable:              api_delta   R-squared:                       0.938
Model:                            OLS   Adj. R-squared:                  0.503
Method:                 Least Squares   F-statistic:                     2.156
Date:                Sun, 02 May 2021   Prob (F-statistic):             0.0358
Time:                        21:27:54   Log-Likelihood:                 7.4919
No. Observations:                 138   AIC:                             227.0
Df Residuals:                      17   BIC:                             581.2
Df Model:                         120                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             5.6339      2.73

## Lasso Regression + Mutual Info Regression
---

Lasso regression to select features. Copying down some data work so we can execute multiple iterations

In [140]:
import random
import itertools

from scipy.stats import rankdata

In [153]:
# Create dataframe for Lasso
df_lasso = df.copy()


In [154]:
# PARAMS
# -------------------------------
iters = 5
seeds = random.sample(range(10000), iters)
alpha = np.arange(0.01, 0.16, 0.01).tolist()

iterations = list(itertools.product(*[seeds, alpha]))

# Store Results
feature_coefs = []
model_results = []  # alpha, mse, mae, r^2
predictions = []
target_idx = []
mutual_info_reg = []

In [155]:
for i in range(len(iterations)):

    a = iterations[i][1]

    # Split the data (seed based on i)
    X_train, X_test, y_train, y_test = train_test_split(
        df_lasso, 
        df_target['api_delta'],
        test_size=0.25,
        random_state = iterations[i][0]
    )

    # Normalize data based on training
    scaler = MinMaxScaler()
    scaler.fit(X_train)

    # Transform training data
    X_train_scale = pd.DataFrame(scaler.transform(X_train))
    X_train_scale.columns = X_train.columns

    # Transform test data
    X_test_scale = pd.DataFrame(scaler.transform(X_test))
    X_test_scale.columns = X_test.columns

    # -----------------
    # MODELING STARTS HERE
    # -----------------
    clf = linear_model.Lasso(alpha = a)
    model = clf.fit(X_train_scale, y_train)

    # Mutual Information Regression
    mir = SelectKBest(score_func=mutual_info_regression, k='all')
    mir.fit(X_train_scale, y_train);
    
    # Predictions
    prediction_lasso = model.predict(X_test_scale)

    # model metrics
    mse = mean_squared_error(y_test, prediction_lasso)
    mae = mean_absolute_error(y_test, prediction_lasso)
    r2 = r2_score(y_test, prediction_lasso)

    # Store details
    feature_coefs.append(model.coef_)
    model_results.append([a, mse, mae, r2])
    predictions.append(list(prediction_lasso))
    target_idx.append(list(y_test.index))
    mutual_info_reg.append(mir.scores_)

In [156]:
# Ranking Importance
df_lasso_results = pd.DataFrame(
    model_results,
    columns = ['alpha', 'mse', 'mae', 'r2']
)

# Feature Coefficiencts
df_lasso_coefs = pd.DataFrame(
    feature_coefs,
    columns = X_train.columns
)

df_mir = pd.DataFrame(
    mutual_info_reg,
    columns = X_train.columns
)

# Combine them!
df_results = pd.concat([df_lasso_results, df_lasso_coefs], axis=1)
df_mir = pd.concat([df_lasso_results, df_mir], axis=1)

In [157]:
# BEST LASSO RESULTS --- alpha is 0.03 or 0.04
df_results[['alpha', 'mae', 'r2']].groupby('alpha').agg(['mean', 'median', 'min']).reset_index().head().round(3)

alpha    mae                   r2              
          mean median    min   mean median    min
0  0.01  0.743  0.739  0.724  0.101  0.035 -0.033
1  0.02  0.724  0.717  0.693  0.169  0.144  0.082
2  0.03  0.711  0.708  0.674  0.196  0.184  0.127
3  0.04  0.724  0.716  0.669  0.176  0.168  0.147
4  0.05  0.739  0.729  0.669  0.147  0.147  0.123

#### Repeating With Alpha = 0.03
---

Repating same framework as above (copied the same code) but with **alpha set to 0.03**<br>
This is what had the best performance!

We've increased the number of iterations to 200

In [158]:
# PARAMS
# -------------------------------

iters = 2000
seeds = random.sample(range(10000), iters)
# alpha = np.arange(0.01, 0.16, 0.01).tolist()
alpha = [0.03]

iterations = list(itertools.product(*[seeds, alpha]))

# Store Results
feature_coefs = []
model_results = []  # alpha, mse, mae, r^2
predictions = []
target_idx = []
mutual_info_reg = []

In [159]:
for i in range(len(iterations)):
    
    # alpha for the iteration
    a = iterations[i][1]

    # Split the data (seed based on i)
    X_train, X_test, y_train, y_test = train_test_split(
        df_lasso, 
        df_target['api_delta'],
        test_size=0.25,
        random_state = iterations[i][0]
    )

    # Normalize data based on training
    scaler = MinMaxScaler()
    scaler.fit(X_train)

    # Transform training data
    X_train_scale = pd.DataFrame(scaler.transform(X_train))
    X_train_scale.columns = X_train.columns

    # Transform test data
    X_test_scale = pd.DataFrame(scaler.transform(X_test))
    X_test_scale.columns = X_test.columns

    # -----------------
    # MODELING STARTS HERE
    # -----------------
    clf = linear_model.Lasso(alpha = a)
    model = clf.fit(X_train_scale, y_train)
    prediction_lasso = model.predict(X_test_scale)

    # Mutual Information Regression
    mir = SelectKBest(score_func=mutual_info_regression, k='all')
    mir.fit(X_train_scale, y_train);

    # model metrics
    mse = mean_squared_error(y_test, prediction_lasso)
    mae = mean_absolute_error(y_test, prediction_lasso)
    r2 = r2_score(y_test, prediction_lasso)

    # Store details
    feature_coefs.append(model.coef_)
    model_results.append([a, mse, mae, r2])
    predictions.append(list(prediction_lasso))
    target_idx.append(list(y_test.index))
    mutual_info_reg.append(mir.scores_)

In [160]:
# Ranking Importance
df_lasso_results = pd.DataFrame(
    model_results,
    columns = ['alpha', 'mse', 'mae', 'r2']
)

# Feature Coefficiencts
df_lasso_coefs = pd.DataFrame(
    feature_coefs,
    columns = X_train.columns
)

# Mutual Information Regression
df_mir = pd.DataFrame(
    mutual_info_reg,
    columns = X_train.columns
)

# Combine them!
df_results = pd.concat([df_lasso_results, df_lasso_coefs], axis=1)
df_mir = pd.concat([df_lasso_results, df_mir], axis=1)

### Logistic Regression Results

In [161]:
## Logistc Regression Results
df_results[['mae', 'r2']].describe()

,mae,r2
count,2000.000000,2000.000000
mean,0.691650,0.140453
std,0.064494,0.092689
min,0.509718,-0.378066
25%,0.647471,0.091102
50%,0.690732,0.154936
75%,0.736046,0.205454
max,0.953673,0.342623


In [194]:
# Feature Importance
# On average, only ~12 feaures are in each Logistic Regression Model!!
(df_lasso_coefs != 0).sum(axis=1).mean()

11.847

In [203]:
# Feature Importance - How often does each feature have a non-zero weight
# Printing the top 30!

count_feature_has_coef = (df_lasso_coefs != 0).sum(axis=0)
api_top_20 = (count_feature_has_coef / df_lasso_coefs.shape[0]).sort_values(ascending = False)[:20]
api_top_20

api1_txfap           0.9970
api2_txfap           0.9960
api3_txfap           0.9275
cdi5_txfap           0.8945
sex                  0.7560
cdi10_txfap          0.7110
cdi9_txfap           0.6840
cdi8_txfap           0.4715
pbq14_txfap          0.4615
csi8_txfap           0.3680
promis_anx6_txfap    0.3315
cdi23_txfap          0.2830
pcsc10_txfap         0.2160
csi14_txfap          0.2015
prsleep3_txfap       0.1915
cdi25_txfap          0.1845
cdi19_txfap          0.1715
prsleep7_txfap       0.1630
csi19_txfap          0.1600
pbq12_txfap          0.1535
dtype: float64

In [204]:
# Feature Importance - How are they ranked based on Mutual Information Regression
mir_top_20 = df_mir.iloc[:, 4:].mean().sort_values(ascending=False)[:20]
mir_top_20

api2_txfap           0.143124
api3_txfap           0.126913
csi15_txfap          0.112312
pbq2_txfap           0.111083
api1_txfap           0.109738
csi16_txfap          0.099148
cdi22_txfap          0.093334
cdi12_txfap          0.083830
pcsc12_txfap         0.074329
pcsc9_txfap          0.073086
cdi13_txfap          0.059683
pcsc10_txfap         0.058413
fdi8_txfap           0.057237
fdi12_txfap          0.055274
csi20_txfap          0.055032
promis_anx1_txfap    0.054405
pbq5_txfap           0.053447
csi7_txfap           0.052195
pri12_txfap          0.046485
cdi16_txfap          0.045525
dtype: float64

In [205]:
# Overlap between MIR and Lasso
print(f'Overlap -- Lasso in MIR: {(api_top_20.index.isin(mir_top_20.index)).sum()}')
print(f'Overlap -- MIR in Lasso: {(mir_top_20.index.isin(api_top_20.index)).sum()}')

Overlap -- Lasso in MIR: 4
Overlap -- MIR in Lasso: 4


In [165]:
# ~42% of the time, there is no information gain from study group
df_mir['study_group'][df_mir['study_group'] == 0].count() / 2000

0.418

In [166]:
# When it is there, its a very low-ranked feature
df_mir['study_group'][df_mir['study_group'] != 0].describe().apply(lambda x: format(x, 'f'))

count    1164.000000
mean        0.030104
std         0.022505
min         0.000028
25%         0.012634
50%         0.026120
75%         0.042402
max         0.127712
Name: study_group, dtype: object

## Logistic Regression - Remaining Features
---

Because we are seeing that each logistic regression is reducing the features pace to only ~12 features on average for `api_delta', let's see how that compares for the other two features.

If there is a lot of overlap between top features we might be able to reduce the number of features quite a bit with little impact to overall performance

In [168]:
# PARAMS (1k Iterations, alpha = 0.03)
# -------------------------------

iters = 1000
seeds = random.sample(range(10000), iters)
alpha = [0.03]

iterations = list(itertools.product(*[seeds, alpha]))

In [169]:
## CSIGI BLOCK
feature_coefs_csigi = []
model_results_csigi = []  # alpha, mse, mae, r^2
predictions_csigi = []
target_idx_csigi = []

for i in range(len(iterations)):

    a = iterations[i][1]

    # Split the data (seed based on i)
    X_train, X_test, y_train, y_test = train_test_split(
        df_lasso, 
        df_target['csigi_delta'],
        test_size=0.25,
        random_state = iterations[i][0]
    )

    # Normalize data based on training
    scaler = MinMaxScaler()
    scaler.fit(X_train)

    # Transform training data
    X_train_scale = pd.DataFrame(scaler.transform(X_train))
    X_train_scale.columns = X_train.columns

    # Transform test data
    X_test_scale = pd.DataFrame(scaler.transform(X_test))
    X_test_scale.columns = X_test.columns

    # -----------------
    # MODELING STARTS HERE
    # -----------------
    clf = linear_model.Lasso(alpha = a)
    model = clf.fit(X_train_scale, y_train)
    prediction_lasso = model.predict(X_test_scale)

    # model metrics
    mse = mean_squared_error(y_test, prediction_lasso)
    mae = mean_absolute_error(y_test, prediction_lasso)
    r2 = r2_score(y_test, prediction_lasso)

    # Store details
    feature_coefs_csigi.append(model.coef_)
    model_results_csigi.append([a, mse, mae, r2])
    predictions_csigi.append(list(prediction_lasso))
    target_idx_csigi.append(list(y_test.index))


In [170]:
## PROMIS PAIN BLOCK
feature_coefs_promis = []
model_results_promis = []  # alpha, mse, mae, r^2
predictions_promis = []
target_idx_promis = []
mutual_info_reg_promis = []

for i in range(len(iterations)):

    a = iterations[i][1]

    # Split the data (seed based on i)
    X_train, X_test, y_train, y_test = train_test_split(
        df_lasso, 
        df_target['api_delta'],
        test_size=0.25,
        random_state = iterations[i][0]
    )

    # Normalize data based on training
    scaler = MinMaxScaler()
    scaler.fit(X_train)

    # Transform training data
    X_train_scale = pd.DataFrame(scaler.transform(X_train))
    X_train_scale.columns = X_train.columns

    # Transform test data
    X_test_scale = pd.DataFrame(scaler.transform(X_test))
    X_test_scale.columns = X_test.columns

    # -----------------
    # MODELING STARTS HERE
    # -----------------
    clf = linear_model.Lasso(alpha = a)
    model = clf.fit(X_train_scale, y_train)
    prediction_lasso = model.predict(X_test_scale)

    # model metrics
    mse = mean_squared_error(y_test, prediction_lasso)
    mae = mean_absolute_error(y_test, prediction_lasso)
    r2 = r2_score(y_test, prediction_lasso)

    # Store details
    feature_coefs_promis.append(model.coef_)
    model_results_promis.append([a, mse, mae, r2])
    predictions_promis.append(list(prediction_lasso))
    target_idx_promis.append(list(y_test.index))

### Results - CSI GI Symptoms

In [179]:
# CSI-GI Results (MAE)
df_csigi_results = pd.DataFrame(
    model_results_csigi,
    columns = ['alpha', 'mse', 'mae', 'r2']
)

# Feature Coefficiencts
df_coefs_csigi = pd.DataFrame(
    feature_coefs_csigi,
    columns = X_train.columns
)

df_csigi_results[['mae', 'r2']].describe()

,mae,r2
count,1000.000000,1000.000000
mean,0.518699,0.163016
std,0.053127,0.081624
min,0.304568,-0.240176
25%,0.484680,0.123180
50%,0.518095,0.177009
75%,0.553594,0.219166
max,0.697730,0.371757


In [196]:
# Feature Importance
print('Average number of features used in each Logistic regression model') 
(df_coefs_csigi != 0).sum(axis=1).mean()

Average number of features used in each Logistic regression model


10.142

In [201]:
# Top 30 features from CSIGI
count_feature_has_coef = (df_coefs_csigi != 0).sum(axis=0)
csigi_top_20 = (count_feature_has_coef / df_coefs_csigi.shape[0]).sort_values(ascending = False)[:20]
csigi_top_20

csi21_txfap          1.000
csi12_txfap          0.967
csi15_txfap          0.953
csi14_txfap          0.904
cdi13_txfap          0.786
cdi10_txfap          0.717
sex                  0.701
cdi19_txfap          0.546
csi13_txfap          0.459
csi22_txfap          0.326
cdi7_txfap           0.313
csi20_txfap          0.288
study_group          0.259
cdi8_txfap           0.252
pcsc11_txfap         0.246
cdi21_txfap          0.156
csi3_txfap           0.144
cdi1_txfap           0.121
promis_anx6_txfap    0.110
promis_anx5_txfap    0.101
dtype: float64

### Results - PROMIS Pain Symptoms

In [183]:
# CSI-GI Results (MSE)
df_promis_results = pd.DataFrame(
    model_results_promis,
    columns = ['alpha', 'mse', 'mae', 'r2']
)

# Feature Coefficiencts
df_coefs_promis = pd.DataFrame(
    feature_coefs_promis,
    columns = X_train.columns
)

df_promis_results[['mse', 'mae', 'r2']].describe()

,mse,mae,r2
count,1000.000000,1000.000000,1000.000000
mean,0.730198,0.692686,0.143508
std,0.118764,0.063913,0.092872
min,0.370167,0.506037,-0.327527
25%,0.645731,0.648575,0.097698
50%,0.724497,0.691759,0.159436
75%,0.808983,0.735459,0.206890
max,1.159192,0.882344,0.337685


In [198]:
# Feature Importance
print('Average number of features used in each Logistic regression model') 
(df_coefs_promis != 0).sum(axis=1).mean()

Average number of features used in each Logistic regression model


11.81

In [202]:
# Top 30 features from PROMIS
count_feature_has_coef = (df_coefs_promis != 0).sum(axis=0)
promis_top_20 = (count_feature_has_coef / df_coefs_promis.shape[0]).sort_values(ascending = False)[:20]
promis_top_20

api1_txfap           0.998
api2_txfap           0.997
api3_txfap           0.933
cdi5_txfap           0.876
sex                  0.731
cdi10_txfap          0.706
cdi9_txfap           0.682
cdi8_txfap           0.473
pbq14_txfap          0.466
promis_anx6_txfap    0.358
csi8_txfap           0.355
cdi23_txfap          0.267
pcsc10_txfap         0.221
csi14_txfap          0.210
prsleep7_txfap       0.204
prsleep3_txfap       0.187
cdi25_txfap          0.184
cdi19_txfap          0.161
pbq12_txfap          0.161
csi19_txfap          0.161
dtype: float64

## Logistic Regression - Only Top 10 Features
---

In [224]:
features = list(api_top_20.index.values)
features.extend(csigi_top_20.index.values)
features.extend(promis_top_20.index.values)

# Reduce to unique (only 34)
features = list(set(features))

In [226]:
# PARAMS (1k Iterations, alpha = 0.03)
# -------------------------------
iters = 1000
seeds = random.sample(range(10000), iters)
alpha = [0.03]

iterations = list(itertools.product(*[seeds, alpha]))

In [229]:
outcome_var = ['api_delta', 'csigi_delta', 'cprinter_delta']
df_lasso_reduced = df_lasso[features] # reduce to our 34 features


feature_coefs_final = []
model_results_final = []  # mse, mae
predictions_final = []

# Same loop as before, but for each target
for target in outcome_var:
    for i in range(len(iterations)):
        
        # iteration alpha
        a = iterations[i][1]

        # Split the data (seed based on i)
        X_train, X_test, y_train, y_test = train_test_split(
            df_lasso, 
            df_target[target],
            test_size=0.25,
            random_state = iterations[i][0]
        )

        # Normalize data based on training
        scaler = MinMaxScaler()
        scaler.fit(X_train)

        # Transform training data
        X_train_scale = pd.DataFrame(scaler.transform(X_train))
        X_train_scale.columns = X_train.columns

        # Transform test data
        X_test_scale = pd.DataFrame(scaler.transform(X_test))
        X_test_scale.columns = X_test.columns

        # -----------------
        # MODELING STARTS HERE
        # -----------------
        clf = linear_model.Lasso(alpha = a)
        model = clf.fit(X_train_scale, y_train)
        prediction_lasso = model.predict(X_test_scale)

        # model metrics
        mse = mean_squared_error(y_test, prediction_lasso)
        mae = mean_absolute_error(y_test, prediction_lasso)

        # Store details
        feature_coefs_final.append(model.coef_)
        model_results_final.append([mse, mae])
        predictions_final.append(list(prediction_lasso))

In [235]:
# CSI-GI Results (MAE)
df_results_final = pd.DataFrame(
    model_results_final,
    columns = ['mse', 'mae']
)

print('API Results')
print(df_results_final[:1000].mean())

print('\nCSIGI Results')
print(df_results_final[1000:2000].mean())

print('\nPROMIS Results')
print(df_results_final[2000:].mean())

API Results
mse    0.722068
mae    0.689555
dtype: float64

CSIGI Results
mse    0.429253
mae    0.520293
dtype: float64

PROMIS Results
mse    72.453160
mae     6.467261
dtype: float64


In [237]:
# Feature Coefficiencts
df_coefs_final = pd.DataFrame(
    feature_coefs_final,
    columns = X_train.columns
)

In [250]:
print('API Top 10 (absolute value)')
df_coefs_final[:1000].mean().abs().sort_values(ascending=False)[:10]

API Top 10 (absoulte value)


api2_txfap           0.556119
api1_txfap           0.456450
api3_txfap           0.274890
cdi5_txfap           0.229689
cdi9_txfap           0.085321
cdi10_txfap          0.081388
cdi8_txfap           0.070331
sex                  0.067440
pbq14_txfap          0.042464
promis_anx6_txfap    0.038185
dtype: float64

In [252]:
print('CSI GI Top 10 (absolute value)')
df_coefs_final[1000:2000].mean().abs().sort_values(ascending=False)[:10]

CSI GI Top 10 (absolute value)


csi21_txfap    0.455350
csi15_txfap    0.222690
csi12_txfap    0.219587
csi14_txfap    0.148602
cdi13_txfap    0.088650
cdi10_txfap    0.071913
sex            0.052579
cdi19_txfap    0.043887
csi13_txfap    0.038172
cdi7_txfap     0.025215
dtype: float64

In [253]:
print('PROMIS Top 10 (absolute value)')
df_coefs_final[2000:].mean().abs().sort_values(ascending=False)[:10]

PROMIS Top 10 (absolute value)


cdi1_txfap     5.957673
pbq15_txfap    5.291473
cdi24_txfap    4.922545
cdi25_txfap    4.669209
csi18_txfap    4.399131
cdi22_txfap    4.022724
fdi8_txfap     3.777352
cdi23_txfap    3.610868
fdi9_txfap     3.079780
pbq10_txfap    2.959566
dtype: float64